In [1]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import pickle
import os


In [2]:
images = {}

COCO = "/Users/mihnea/_workspace_/_uni/workshop/BOLD5000/Scene_Stimuli/Presented_Stimuli/COCO"  #directory with the COCO presented stimuli
ImageNet = "/Users/mihnea/_workspace_/_uni/workshop/BOLD5000/Scene_Stimuli/Presented_Stimuli/ImageNet" #directory with the ImageNet presented stimuli
Scene = "/Users/mihnea/_workspace_/_uni/workshop/BOLD5000/Scene_Stimuli/Presented_Stimuli/Scene"   #directory with the Scene presented stimuli

paths = [COCO, ImageNet, Scene]

for path in paths:
    for filename in os.listdir(path):
        #print(filename)
        img_path = os.path.join(path, filename)
        img = Image.open(img_path)
        images[filename] = img
    

In [3]:
for image in images:
    print (images[image])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x169804310>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x11041F5D0>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x169829AD0>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x169829B10>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x169829E10>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x16982A1D0>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x16982A590>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x16982A950>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x16982AD10>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x16982B0D0>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x16982B490>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x375 at 0x16982B850>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB si

In [4]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

/Users/mihnea/_workspace_/_uni/workshop/fmri_data/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [5]:
one_dim_embeddings = {}
max_length = 10

for image in images:
    if len(one_dim_embeddings) == max_length:
        break
    else:
        inputs = processor(images=images[image], return_tensors="pt")
        outputs = model(**inputs)
        one_dim_embeddings[image] = (outputs.last_hidden_state).reshape(-1)

In [6]:
#saves the one dimensional embeddings as a .pkl file in the current working directory
embeddings_base_folder = "/Users/mihnea/_workspace_/_uni/workshop/BOLD5000/embeddings"  # Base folder 
dataset_subfolders = ["COCO", "ImageNet", "Scene"]  # Names of dataset subfolders

# Create folders if they don't exist
for subfolder in dataset_subfolders:
    folder_path = os.path.join(embeddings_base_folder, subfolder)
    os.makedirs(folder_path, exist_ok=True)

# Save embeddings with proper naming and absolute paths
for image, embedding in one_dim_embeddings.items():
    # Determine the dataset subfolder based on the image name
    for subfolder in dataset_subfolders:
        if subfolder in image:  
            break  # Exit the loop once the subfolder is found

    embedding_filename = "embedding_" + image + ".pkl"
    embedding_path = os.path.join(embeddings_base_folder, subfolder, embedding_filename)

    with open(embedding_path, 'wb') as emb:
        pickle.dump(embedding, emb)